In [15]:
import pandas as pd
df_clean = pd.read_parquet("../data/interim/final_clean.parquet")
df_clean.head()


,row_name,clip_start,clip_end,set_num,set_num:_time,set_p1,set_p1:_time,set_p2,set_p2:_time,servicio,...,golpe_q:_time,fin_golpe,fin_golpe:_time,fin_golpe:_x,fin_golpe:_y,inicio_golpe,inicio_golpe:_time,inicio_golpe:_x,inicio_golpe:_y,__source_file
0,1º Servicio,108.058,108.788,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Arturo Coello,...,108.058,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Roterdam_CoelloTapia_NietoYanguas CSV.csv
1,1º Servicio,131.279,132.01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Arturo Coello,...,131.279,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Roterdam_CoelloTapia_NietoYanguas CSV.csv
2,1º Servicio,162.04,162.77,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Arturo Coello,...,162.04,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Roterdam_CoelloTapia_NietoYanguas CSV.csv
3,1º Servicio,194.415,195.146,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Arturo Coello,...,194.415,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Roterdam_CoelloTapia_NietoYanguas CSV.csv
4,1º Servicio,214.437,215.167,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Arturo Coello,...,214.437,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Roterdam_CoelloTapia_NietoYanguas CSV.csv


In [16]:
df_clean.to_excel("coordenadas_limpias.xlsx", index=False)

PermissionError: [Errno 13] Permission denied: 'coordenadas_limpias.xlsx'

In [17]:
import pandas as pd
import os
import plotly.graph_objects as go
import numpy as np 

# ======================================================
# CONSTANTES (Simulación de una cancha 10x20)
# ======================================================

ANCHO_PISTA = 100
LARGO_PISTA = 200

COL_JUGADOR  = "jugador"

# === NOMBRES DE COLUMNAS PREDETERMINADOS Y ACTUALIZADOS ===
# (He cambiado COL_INICIO_X por el typo 'gople' para coincidir con tu DataFrame de prueba)
COL_INICIO_X = "inicio_gople:_x"  
COL_INICIO_Y = "inicio_golpe:_y"  
COL_FIN_X    = "fin_golpe:_x"     # Asumiendo que esta es correcta
COL_FIN_Y    = "fin_golpe:_y"     # Asumiendo que esta es correcta
COL_WINNER   = "winner"
COL_ERROR    = "error"
# ==========================================================

COLORES_EVENTO = {
    "winner": "#00BFFF",
    "error no forzado": "#FF9800",
    "missed": "#FF3B3B",
    "bola dentro": "#4CAF50"
}

In [29]:
# ======================================================
# FUNCIONES DE PREPROCESAMIENTO
# ======================================================

def detectar_columnas_coordenadas(df):
    """
    Detecta automáticamente los nombres correctos de columnas
    para inicio_x, inicio_y, fin_x, fin_y, incluso si hay typos como 'gople' o ':'.
    """
    posibles_nombres = {
        "inicio_x": ["inicio_gople:_x", "inicio_gople_x", "inicio_golpe:_x", "inicio_golpe_x"],
        "inicio_y": ["inicio_gople:_y", "inicio_gople_y", "inicio_golpe:_y", "inicio_golpe_y"],
        "fin_x":    ["fin_gople:_x", "fin_gople_x", "fin_golpe:_x", "fin_golpe_x"],
        "fin_y":    ["fin_gople:_y", "fin_gople_y", "fin_golpe:_y", "fin_golpe_y"]
    }

    encontrados = {}
    for clave, lista in posibles_nombres.items():
        for nombre in lista:
            if nombre in df.columns:
                encontrados[clave] = nombre
                break
        if clave not in encontrados:
            encontrados[clave] = None

    print("🧭 Columnas detectadas automáticamente:")
    for k, v in encontrados.items():
        print(f"  {k}: {v}")

    return encontrados

def clasificar_eventos(df):
    """Clasifica los eventos en categorías (winner, error no forzado, missed, bola dentro)."""
    d = df.copy()
    
    # Asegurar que las columnas existan o usar vacías
    for c in [COL_WINNER, COL_ERROR]:
        if c not in d.columns:
            d[c] = ""
        d[c] = d[c].astype(str).str.lower().str.strip()
        
    d["categoria"] = "bola dentro"
    d.loc[d[COL_ERROR].str.contains("error no forzado", na=False), "categoria"] = "error no forzado"
    d.loc[d[COL_ERROR].str.contains("missed", na=False), "categoria"] = "missed"
    d.loc[d[COL_WINNER].str.contains("winner", na=False), "categoria"] = "winner"
    return d

In [ ]:
import plotly.graph_objects as go
import pandas as pd
# Asegúrate de que las constantes COL_JUGADOR, ANCHO_PISTA, LARGO_PISTA, y COLORES_EVENTO 
# estén definidas y ejecutadas.

def pintar_pista_interactiva(df, inicio_x_col, inicio_y_col, fin_x_col, fin_y_col):
    """
    Genera y EXPORTA un gráfico interactivo (Plotly) de las trayectorias de golpes.
    Esta versión es a PRUEBA DE FALLOS contra el ValueError, usando solo la opacidad 
    del marcador para la ocultación.
    """
    jugadores = df.get(COL_JUGADOR, pd.Series()).dropna().unique().tolist()
    
    if not jugadores:
        print("⚠️ No se encontraron jugadores válidos en la columna 'jugador'.")
        return

    print(f"👥 Jugadores detectados: {jugadores}")
    
    for jugador in jugadores:
        sub = df[df[COL_JUGADOR] == jugador].dropna(subset=[inicio_x_col, inicio_y_col, fin_x_col, fin_y_col])
        
        if sub.empty:
            print(f"⚠️ Sin coordenadas válidas para {jugador}, no se genera pista ni HTML.")
            continue

        fig = go.Figure()

        # --- DIBUJO DE LA PISTA DE PÁDEL ---
        SERVICIO_Y_OFFSET = 60 
        CENTRO_X = ANCHO_PISTA / 2
        
        # Formas de la pista
        fig.add_shape(type="rect", x0=0, y0=0, x1=ANCHO_PISTA, y1=LARGO_PISTA, line=dict(color="white", width=3)) 
        fig.add_shape(type="line", x0=0, y0=LARGO_PISTA/2, x1=ANCHO_PISTA, y1=LARGO_PISTA/2, line=dict(color="white", width=4)) 
        fig.add_shape(type="line", x0=0, y0=LARGO_PISTA/2 - SERVICIO_Y_OFFSET, x1=ANCHO_PISTA, y1=LARGO_PISTA/2 - SERVICIO_Y_OFFSET, line=dict(color="white", width=2))
        fig.add_shape(type="line", x0=0, y0=LARGO_PISTA/2 + SERVICIO_Y_OFFSET, x1=ANCHO_PISTA, y1=LARGO_PISTA/2 + SERVICIO_Y_OFFSET, line=dict(color="white", width=2))
        fig.add_shape(type="line", x0=CENTRO_X, y0=LARGO_PISTA/2, x1=CENTRO_X, y1=LARGO_PISTA/2 - SERVICIO_Y_OFFSET, line=dict(color="white", width=2))
        fig.add_shape(type="line", x0=CENTRO_X, y0=LARGO_PISTA/2, x1=CENTRO_X, y1=LARGO_PISTA/2 + SERVICIO_Y_OFFSET, line=dict(color="white", width=2))
        
        # --- DIBUJO DE TRAYECTORIAS (SIN FLECHAS) ---
        traces_to_add = []
        
        for cat, color in COLORES_EVENTO.items():
            df_cat = sub[sub["categoria"] == cat].reset_index() 
            if df_cat.empty: continue
                
            for i, r in df_cat.iterrows():
                # 1. Traza de línea y marcador 
                traces_to_add.append(go.Scatter(
                    x=[r[inicio_x_col], r[fin_x_col]], 
                    y=[r[inicio_y_col], r[fin_y_col]], 
                    mode="lines+markers",
                    line=dict(color=color, width=2),
                    marker=dict(size=8, symbol='triangle-up'),
                    showlegend=bool(i == df_cat.index[0]), 
                    name=cat,
                    hovertext=f"{cat} ({jugador})",
                    hoverinfo="text",
                    legendgroup=cat,
                    
                    # --- CÓDIGO A PRUEBA DE FALLOS: SOLO OPACIDAD DEL MARCADOR ---
                    unselected=dict(
                        marker=dict(opacity=0) 
                        # Eliminamos la línea para evitar el "Bad property path: line"
                    )
                    # -------------------------------------------------------------
                ))

        fig.add_traces(traces_to_add)
        
        # --- CONFIGURACIÓN VISUAL, TAMAÑO Y LEYENDA ---
        fig.update_layout(
            title=f"Pista de Pádel — {jugador} (Ocultación Total al Clic)",
            plot_bgcolor="#003C77", 
            paper_bgcolor="#00244D",
            xaxis=dict(visible=False, range=[0, ANCHO_PISTA]),
            yaxis=dict(visible=False, range=[LARGO_PISTA, 0], scaleanchor="x", scaleratio=1), 
            font=dict(color="white"),
            showlegend=True,
            legend=dict(
                groupclick="togglegroup", 
                itemclick="toggle"   
            ),
            height=900,  
            width=450 
        )
        
        # --- EXPORTACIÓN HTML (Responsiva) ---
        nombre_archivo = f"pista_padel_{jugador.lower()}_ocultacion_total.html"
        
        fig.write_html(
            nombre_archivo, 
            auto_open=False,
            full_html=True,
        ) 
        
        print(f"✅ Gráfico guardado como {nombre_archivo}")

In [54]:
# ======================================================
# EJECUCIÓN DE PRUEBA A GRAN ESCALA (500 GOLPES)
# ======================================================

NUM_GOLPES = 50
np.random.seed(42) # Para reproducibilidad

# 1. Crear un DataFrame grande
# Simulación de jugadores y eventos
jugadores_sim = np.random.choice(["Nadal", "Federer", "Djokovic", "Alcaraz"], NUM_GOLPES)
eventos_sim = np.random.choice(["winner", "error no forzado", "missed", ""], NUM_GOLPES, p=[0.1, 0.2, 0.1, 0.6])

df_grande = pd.DataFrame({
    COL_JUGADOR: jugadores_sim,
    COL_WINNER: np.where(eventos_sim == "winner", "winner", ""),
    COL_ERROR: np.where(eventos_sim != "winner", eventos_sim, ""),
    
    # Coordenadas X: Entre 5 y 95 (casi nunca en los extremos)
    COL_INICIO_X: np.random.uniform(5, 95, NUM_GOLPES), 
    COL_FIN_X: np.random.uniform(5, 95, NUM_GOLPES), 
    
    # Coordenadas Y de INICIO: Campo propio (110 a 195)
    COL_INICIO_Y: np.random.uniform(110, 195, NUM_GOLPES), 
    
    # Coordenadas Y de FIN: Campo contrario (5 a 90)
    COL_FIN_Y: np.random.uniform(5, 90, NUM_GOLPES)
})

# 2. Clasificar eventos (añade la columna 'categoria')
df_grande = clasificar_eventos(df_grande)

# 3. Detectar nombres de columnas
col_map = detectar_columnas_coordenadas(df_grande)

# 4. Asignar los nombres detectados
inicio_x_col = col_map.get("inicio_x", COL_INICIO_X)
inicio_y_col = col_map.get("inicio_y", COL_INICIO_Y)
fin_x_col    = col_map.get("fin_x", COL_FIN_X)
fin_y_col    = col_map.get("fin_y", COL_FIN_Y)

print(f"\n✅ Columnas finales para Plotly: Inicio=({inicio_x_col}, {inicio_y_col}), Fin=({fin_x_col}, {fin_y_col})")

# 5. Generar visualizaciones (Esto tomará más tiempo)
pintar_pista_interactiva(df_grande, inicio_x_col, inicio_y_col, fin_x_col, fin_y_col)

🧭 Columnas detectadas automáticamente:
  inicio_x: inicio_gople:_x
  inicio_y: inicio_golpe:_y
  fin_x: fin_golpe:_x
  fin_y: fin_golpe:_y

✅ Columnas finales para Plotly: Inicio=(inicio_gople:_x, inicio_golpe:_y), Fin=(fin_golpe:_x, fin_golpe:_y)
👥 Jugadores detectados: ['Djokovic', 'Alcaraz', 'Nadal', 'Federer']
✅ Gráfico guardado como pista_padel_djokovic_ocultacion_total.html
✅ Gráfico guardado como pista_padel_alcaraz_ocultacion_total.html
✅ Gráfico guardado como pista_padel_nadal_ocultacion_total.html
✅ Gráfico guardado como pista_padel_federer_ocultacion_total.html
